In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [42]:
from keras.applications.vgg19 import Vgg19
from keras.models import Sequential, Model
from keras.layers import Input
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from sklearn.metrics import fbeta_score
import numpy as np

ImportError: cannot import name Vgg19

In [3]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import ama as a
TrainBatch = a.trainbatch.TrainBatch

In [9]:
cv_version = str(1)

In [38]:
path= '../data/sample'
batch_size = 128
img_size = (128,128)

In [32]:
??ResNet50

In [39]:
res = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(3,)+img_size))

In [41]:
res.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_12 (InputLayer)            (None, 3, 128, 128)   0                                            
____________________________________________________________________________________________________
zeropadding2d_11 (ZeroPadding2D) (None, 3, 134, 134)   0           input_12[0][0]                   
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 64, 64, 64)    9472        zeropadding2d_11[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 64, 64, 64)    128         conv1[0][0]                      
___________________________________________________________________________________________

In [40]:
x = res.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
preds = Dense(17, activation='sigmoid')(x)

model = Model(inputs=res.input, output=preds)
for layer in model.layers:
    layer.trainable = False
model.layers[-1].trainable = True

Exception: The shape of the input to "Flatten" is not fully defined (got (2048, 0, 0). Make sure to pass a complete "input_shape" or "batch_input_shape" argument to the first layer in your model.

In [ ]:
valgen = TrainBatch(path+'val-jpg'+cv_version+'/', path+'train_v2.csv', batch_size=batch_size, img_size=img_size)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(rotation_range=5, horizontal_flip=True, vertical_flip=True)

traingen = TrainBatch(path+'train-jpg'+cv_version+'/', path+'train_v2.csv', batch_size=batch_size, img_size=img_size,
                      imagegen=gen)

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from ama.persistenthistory import PersistentHistory

earlystop = EarlyStopping(patience=4)
history = PersistentHistory()

In [ ]:
filepath = '../data/weights/resnet'+cv_version+'_best_lr1.hk'
saver = ModelCheckpoint(filepath, verbose=1, save_best_only=True, save_weights_only=True)
vgg.model.compile(optimizer=Adam(lr=0.001),loss='binary_crossentropy', metrics=['accuracy'])
vgg.model.fit_generator(traingen, samples_per_epoch=traingen.nb_sample, nb_epoch=20,
                        validation_data=valgen, nb_val_samples=valgen.nb_sample,
                        callbacks=[history, saver, earlystop])